            # PhysAE – démarrage rapide entraînement

            Ce carnet montre comment préparer les données, instancier le modèle et
            enchaîner les trois stages d'entraînement avec des configurations
            allégées pour une exécution locale rapide.
            


            ## 1. Installation et import

            Assurez-vous d'être dans un environnement où PhysAE est installé en
            mode développement (`pip install -e .[docs]`).
            Les importations ci-dessous centralisent tous les modules utilisés
            dans le carnet.
            


In [ ]:
            from pathlib import Path

            from physae.factory import build_data_and_model
            from physae.training import train_stage_A, train_stage_B1, train_stage_B2
            from physae.config_loader import merge_dicts

            ARTIFACT_DIR = Path('artifacts/quickstart')
            ARTIFACT_DIR.mkdir(parents=True, exist_ok=True)
            print('Dossier de sortie :', ARTIFACT_DIR.resolve())
            


            ## 2. Chargement et adaptation de la configuration

            Nous réduisons le nombre d'échantillons et la taille des lots pour
            pouvoir tester la boucle d'entraînement en quelques minutes sur CPU.
            


In [ ]:
            data_overrides = {
                'n_train': 1024,
                'n_val': 128,
                'batch_size': 8,
            }
            stageA_overrides = {'epochs': 5, 'enable_progress_bar': True}
            stageB1_overrides = {'epochs': 4, 'enable_progress_bar': True}
            stageB2_overrides = {'epochs': 6, 'enable_progress_bar': True}

            model, train_loader, val_loader = build_data_and_model(config_overrides=data_overrides)
            print(f'Train loader: {len(train_loader)} batches - batch_size={train_loader.batch_size}')
            print(f'Val loader: {len(val_loader)} batches')
            


            ## 3. Exécution des stages

            Chaque fonction `train_stage_*` applique les réglages définis dans
            `physae/configs/stages`. Nous pouvons injecter des surcharges
            supplémentaires à l'aide du paramètre `config_overrides`.
            


In [ ]:
            stage_A_cfg = merge_dicts({'config_overrides': stageA_overrides}, {})
            stage_B1_cfg = merge_dicts({'config_overrides': stageB1_overrides}, {})
            stage_B2_cfg = merge_dicts({'config_overrides': stageB2_overrides}, {})

            train_stage_A(model, train_loader, val_loader, ckpt_out=str(ARTIFACT_DIR/'stage_A.ckpt'), **stage_A_cfg)
            train_stage_B1(
                model,
                train_loader,
                val_loader,
                ckpt_in=str(ARTIFACT_DIR/'stage_A.ckpt'),
                ckpt_out=str(ARTIFACT_DIR/'stage_B1.ckpt'),
                **stage_B1_cfg,
            )
            train_stage_B2(
                model,
                train_loader,
                val_loader,
                ckpt_in=str(ARTIFACT_DIR/'stage_B1.ckpt'),
                ckpt_out=str(ARTIFACT_DIR/'stage_B2.ckpt'),
                **stage_B2_cfg,
            )
            


            ## 4. Inspection rapide des métriques

            Après l'entraînement, on peut charger le dernier checkpoint et
            utiliser `evaluate_and_plot` pour visualiser quelques reconstructions.
            


In [ ]:
            from physae.evaluation import evaluate_and_plot
            import torch

            model_ckpt = type(model).load_from_checkpoint(str(ARTIFACT_DIR/'stage_B2.ckpt'))
            model_ckpt.eval()
            if torch.cuda.is_available():
                model_ckpt.cuda()
            metrics_df = evaluate_and_plot(model_ckpt, val_loader, n_show=3, refine=True)
            metrics_df
            
